In [1]:
from google.colab import drive
drive.mount('/content/drive')
import csv
import re
import random
import tqdm.notebook as tq
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 讀入語料與構式辭典

In [2]:
file_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/data/threads/cht-2020-merged.txt'

with open(file_path, 'r', encoding = 'utf-8') as f:
  comments = [ line.strip() for line in f ] # comments 中的每一個 element 是一則評論

In [3]:
construcion_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/sentiment-construction-list/constructions_list/constructions_0406.csv'

with open(construcion_path) as f:
    constructions = [ row.strip() for row in f ]

## 從每則評論中找出符合的構式例子

In [4]:
n = 5  # sample size
random.seed(10)

matched = {}
matched_con = {}
for k in tq.tqdm(constructions):
  pat = re.compile(k)
  if k not in matched: matched[k] = []
  
  # Get all cnstr from all comments
  candidates = set()
  for cmt in comments:
    for c in pat.finditer(cmt): candidates.add(c[0])
  
  # Count all cnstr
  matched_con[k] = len(candidates)

  # Sample
  if len(candidates) > n:
    for c in random.sample(candidates, n): matched[k].append(c)
  else:
    for c in candidates: matched[k].append(c)

In [5]:
import pprint
pprint.pprint(matched)

{'(對\\w+)?一點(都|也)不\\w+\\b': ['一點都不優',
                             '一點都不輸給國外的景色',
                             '一點都不期望5g',
                             '一點都不會煩惱',
                             '一點也不鳥'],
 '(很|也)夠\\w+了\\b': ['也夠測一輪了', '很夠力了', '很夠了應該也不會升就是了', '也夠買一隻平價機了', '也夠打94分鐘了'],
 '(真的)?有夠\\w+的?\\b': ['有夠消極的處理方式',
                      '有夠後悔為了便宜換到亞太',
                      '有夠丟臉的',
                      '有夠虧買空機就好',
                      '有夠多3樓可能會更多'],
 '(置|拉)板凳等\\w+\\b': ['拉板凳等中華', '置板凳等後續'],
 '\\b\\w+(到|的|得)離譜': ['又用手機網路分享限制容量更少到離譜',
                      '一堆辦吃到飽的速度低的離譜',
                      '寫的資料錯的離譜',
                      '慢的離譜',
                      '延遲高得離譜'],
 '\\b\\w+(掰|bye)了': ['彰銀my樂應該真的掰了',
                     '完全跟5g說掰掰了',
                     '新增消費5000門檻就bye了',
                     '過沒半年就掰了',
                     '有前綴代扣字樣就bye了'],
 '\\b\\w+(的|得)要死': ['其他家的訊號都爛得要死',
                    '爛的要死',
                    '台灣之星很多地方訊號都差的要死',
                    '30萬的三大業者怕得要死',
    

## 將符合構式的例子及其數量整理成表格

In [6]:
df = pd.DataFrame([matched_con, matched]).T
df.columns = ['count', 'examples']
df

,count,examples
只有遠傳\s?沒有\w{2},25,"[只有遠傳沒有速度, 只有遠傳沒有朋友, 只有遠傳沒有頑皮, 只有遠傳 沒有網路, 只有遠傳..."
純噓\w+\b,20,"[純噓獲利王, 純噓台灣之星, 純噓3, 純噓亞太, 純噓胎哥獲利王]"
\b\w+能\w\s?，?\w+都能\w了\b,2,"[店員的話能信，連屎都能吃了, 屁話能信史都能吃了]"
八成\w+\b,81,"[八成是沒在關注評測影片, 八成是基地台被抗議拆了, 八成又是台星吧, 八成蠻適合習慣看第四..."
\b\w+秀下限\w+\b,16,"[不懂就不要在那邊秀下限好嗎, 一定會有貪小便宜秀下限想要鑽漏洞的客人樓主說台灣鯛是能占用戶..."
勿忘\w+\b,4,"[勿忘黑心頂新, 勿忘記亞太是以電信服務為主的公司, 勿忘iphone, 勿忘499之亂]"
\b\w+心酸的?\w*\b,78,"[預購心酸的, 好心酸, 手机是买心酸的, 事情傳到樓主爸媽耳裡應該是會有點心酸不管樓主多有..."
\b\w+根本\w{2}\b,292,"[看位置吧女一後棟根本災難, 莊八一樓根本悲劇, 有一門599根本沒救, 老實說這間我覺得根..."
\b\w+敲盤子\w*\b,7,"[遠傳台哥是敲盤子喔, 蘋果又出來敲盤子, 不然台灣人還在被三大敲盤子, 這合約分明敲盤子,..."
\b\w+被當盤子\w*\b,58,"[本來就要要被當盤子的心理準備了, 不然難道要傻傻被當盤子嗎, 如果是的話我就被當盤子了, ..."


In [7]:
# export as csv file
with open('matched_random.csv', 'w') as f:
    for key in matched.keys():
        f.write("%s,%s\n"%(key, matched[key]))